In [1]:
pip install mysqlclient

In [2]:
import sqlalchemy

In [64]:
engine = sqlalchemy.create_engine("mysql+mysqldb://root:" + "12345" + "@15.164.160.109/test"  , encoding='utf-8')

In [6]:
import requests

In [7]:
head = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36"}

In [8]:
url = "https://finance.naver.com/item/sise_day.naver?code=018700"

r = requests.get(url, headers=head)

In [11]:
import pandas as pd

In [65]:
stock = pd.read_html(r.text)[0].dropna()

In [68]:
stock.columns = ['date', 'close', 'before', 'open', 'high', 'low', 'trade']

In [69]:
from datetime import date, datetime

In [70]:
stock['date'] = stock['date'].apply(lambda x : datetime.strptime(x, "%Y.%m.%d"))

In [71]:
days = dict(zip(range(0, 7), [x + "요일" for x in "월화수목금토일"]))

In [72]:
stock['요일'] = stock['date'].apply(lambda x : days[x.weekday()])

In [ ]:
stock.to_sql(name='stock2', con=engine, if_exists='append', index=False)

In [48]:
url_post = "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"

In [49]:
payload = {"bld" : "dbms/MDC/STAT/standard/MDCSTAT01901",
"mktId" : "ALL",
"share" : "1",
"csvxls_isNo" : "false",}

In [50]:
r2 = requests.post(url_post, headers= head, data=payload)

In [55]:
master = pd.DataFrame(r2.json()['OutBlock_1'])

In [74]:
import pymysql

In [76]:
# DB 접속 
try:
    con = pymysql.connect(host="15.164.160.109", user= 'root', port=3306, password= '12345',  
                          db= 'stocks', charset='utf8')
    cur = con.cursor()
except Exception as e:
    print (e)   

In [78]:
master

Index(['ISU_CD', 'ISU_SRT_CD', 'ISU_NM', 'ISU_ABBRV', 'ISU_ENG_NM', 'LIST_DD',
       'MKT_TP_NM', 'SECUGRP_NM', 'SECT_TP_NM', 'KIND_STKCERT_TP_NM', 'PARVAL',
       'LIST_SHRS'],
      dtype='object')

In [79]:
engine = sqlalchemy.create_engine("mysql+mysqldb://root:" + "12345" + "@15.164.160.109/stocks"  , encoding='utf-8')

In [84]:
master.iloc[:0,:]

,ISU_CD,ISU_SRT_CD,ISU_NM,ISU_ABBRV,ISU_ENG_NM,LIST_DD,MKT_TP_NM,SECUGRP_NM,SECT_TP_NM,KIND_STKCERT_TP_NM,PARVAL,LIST_SHRS


In [83]:
master.iloc[:0,:].to_sql(name='day_stock', con=engine, if_exists='append', index=False)

In [87]:
master.columns.size

12

In [98]:
sql = "INSERT INTO day_stock VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s )"

In [101]:
for idx, cont in master.iterrows():
    if idx % 10 == 0: print(idx)
    try:
        cur.execute(sql, cont.values.tolist())
    except Exception as e:
        print (e)
        break

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [100]:
con.commit()

In [105]:
target_master = pd.read_sql_query("select ISU_SRT_CD, ISU_NM from day_stock", con)

In [136]:
target_master.shape

(2585, 2)

In [108]:
target_master[target_master.ISU_NM.str.contains("셀트리온") ]

,ISU_SRT_CD,ISU_NM
986,068270,셀트리온보통주
987,068760,셀트리온제약
988,091990,셀트리온헬스케어


In [112]:
stock_url = "https://finance.naver.com/item/sise_day.naver?code={}&page={}"

In [114]:
total = []
for x in range(1, 101):
    r = requests.get(stock_url.format('091990', x), headers=head)
    total.append(pd.read_html(r.text)[0].dropna())

In [115]:
셀트리온 =pd.concat(total)

In [126]:
셀트리온 = 셀트리온.iloc[::-1].copy()

In [129]:
셀트리온['전날대비'] = 셀트리온['종가'] - 셀트리온['종가'].shift(1)

In [132]:
셀트리온['MA60'] = 셀트리온.종가.rolling(60).mean()

In [138]:
셀트리온.dropna()

,날짜,종가,전일비,시가,고가,저가,거래량,전날대비,MA60
1,2017.12.11,84700.0,6800.0,79000.0,84900.0,78800.0,1922122.0,6800.0,65606.666667
13,2017.12.12,87000.0,2300.0,87300.0,89200.0,84500.0,1917006.0,2300.0,66312.500000
12,2017.12.13,93100.0,6100.0,88700.0,93800.0,87300.0,2573348.0,6100.0,67099.166667
11,2017.12.14,94000.0,900.0,95900.0,97700.0,86700.0,7190220.0,900.0,67880.833333
10,2017.12.15,90500.0,3500.0,93400.0,94600.0,90500.0,1697765.0,-3500.0,68597.500000
...,...,...,...,...,...,...,...,...,...
5,2021.10.01,103600.0,5800.0,108700.0,108800.0,103500.0,1198135.0,-5800.0,115065.000000
4,2021.10.05,90300.0,13300.0,100200.0,100400.0,89900.0,2680190.0,-13300.0,114658.333333
3,2021.10.06,86200.0,4100.0,90300.0,92500.0,86100.0,2304652.0,-4100.0,114191.666667
2,2021.10.07,86700.0,500.0,87400.0,89400.0,85800.0,1331773.0,500.0,113768.333333


In [135]:
셀트리온.columns

Index(['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량', '전날대비', 'MA60'], dtype='object')

대상 : 2585 종목 
table 명 : deabak_jongmok
컬럼 : code, code_name, date, close, before, open, high, low, trade, ma05, ma20, ma60, ma120
기간 : 네이버 수집시 page 100페이지 

주의사항 : NA 삭제, 종복 허용 X 